In [ ]:
%pip install semantic-link
%pip install pyspark
%pip install azure-keyvault-secrets==4.7.0
%pip install azure-identity
%pip install azure-mgmt-resource
%pip install azure-storage-blob
%pip install azure-core

In [ ]:
import pandas as pd
import sempy.fabric as fabric
from sempy.fabric import FabricDataFrame
from sempy.dependencies import plot_dependency_metadata
from sempy.samples import download_synthea

In [ ]:
download_synthea(which='small')

## Explorer les données

In [ ]:
providers = FabricDataFrame(pd.read_csv("synthea/csv/providers.csv"))
providers.head()

In [ ]:
providers.shape

(5855, 12)

### Vérifiez les problèmes de qualité des données avec la fonction find_dependencies de SemPy en traçant un graphique des dépendances fonctionnelles détectées automatiquement :

In [ ]:
deps = providers.find_dependencies()
plot_dependency_metadata(deps)

### Le graphique des dépendances fonctionnelles montre que Id détermine le NOM et l'ORGANISATION (indiqués par les flèches pleines), ce qui est attendu puisque Id est unique :

In [ ]:
providers.Id.is_unique

## Analyser en profondeur les dépendances fonctionnelles

Le graphique des dépendances fonctionnelles montre également que ORGANIZATION détermine ADDRESS et ZIP, comme prévu. Cependant, vous pouvez vous attendre à ce que ZIP détermine également CITY, mais la flèche en pointillé indique que la dépendance n'est qu'approximative, pointant vers un problème de qualité des données.

Il y a d'autres particularités dans le graphique. Par exemple, NOM ne détermine pas le SEXE, l'ID, la SPÉCIALITÉ ou l'ORGANISATION. Chacun de ces éléments mériterait peut-être d’être étudié.

Examinez de plus près la relation approximative entre ZIP et CITY, en utilisant la fonction list_dependency_violations de SemPy pour voir une liste tabulaire des violations :


In [ ]:
providers.list_dependency_violations('ZIP', 'CITY')

In [ ]:
providers.plot_dependency_violations('ZIP', 'CITY')

Le graphique des violations de dépendance affiche les valeurs de ZIP sur le côté gauche et les valeurs de CITY sur le côté droit. Une arête relie un code postal à gauche avec une ville à droite s'il existe une ligne contenant ces deux valeurs. Les bords sont annotés avec le nombre de ces lignes. Par exemple, il y a deux lignes avec le code postal 02747-1242, une ligne avec la ville « NORTH DARTHMOUTH » et l'autre avec la ville « DARTHMOUTH », comme indiqué dans le tracé précédent et le code suivant :

In [ ]:
providers[providers.ZIP == '02747-1242'].CITY.value_counts()

Le graphique montre également que parmi les lignes comportant CITY  "DARTMOUTH", neuf lignes ont un code postal de 02747-1262 ; une ligne a un code postal de 02747-1242  :

In [ ]:
providers[providers.CITY == 'DARTMOUTH'].ZIP.value_counts()


En revenant au graphique des violations de dépendances, vous pouvez voir qu'il existe en fait plusieurs problèmes intéressants de qualité des données présents dans ce modèle sémantique :

Certains noms de villes sont tous en majuscules. Ce problème est facile à résoudre à l’aide de méthodes de chaîne.


Certains noms de villes comportent des qualificatifs (ou préfixes), tels que « Nord » et « Est ». Par exemple, le code postal « 2128 » correspond une fois à « EAST BOSTON » et une fois à « BOSTON ». Un problème similaire se produit entre « NORTH DARTHMOUTH » et « DARTHMOUTH ». Vous pouvez essayer de supprimer ces qualificatifs ou de mapper les codes postaux à la ville avec l'occurrence la plus courante.


Il y a des fautes de frappe dans certaines villes, telles que "PITTSFIELD" contre "PITTSFILED" et "NEWBURGPORT contre "NEWBURYPORT". Dans le cas de "NEWBURGPORT", cette faute de frappe peut être corrigée en utilisant l'occurrence la plus courante. Dans le cas de " PITTSFIELD", n'ayant qu'une seule occurrence chacune, il est beaucoup plus difficile de lever l'ambiguïté automatique sans connaissances externes ni utilisation d'un modèle de langage.
Parfois, les préfixes comme « Ouest » sont abrégés en une seule lettre « W ». Cela pourrait potentiellement être résolu par un simple remplacement, si toutes les occurrences de « W » signifient « Ouest ».


Le code postal "02130" correspond une fois à "BOSTON" et à "Jamaica Plain" une fois. Ce problème n’est pas facile à résoudre, mais s’il y avait plus de données, le mappage vers l’occurrence la plus courante pourrait être une solution potentielle.

## Nettoyer les données

Résolvez les problèmes de majuscules en remplaçant toutes les majuscules par la casse du titre :

In [ ]:
providers['CITY'] = providers.CITY.str.title()

Exécutez à nouveau la détection des violations pour constater que certaines ambiguïtés ont disparu (le nombre de violations est plus petit) :

In [ ]:
providers.list_dependency_violations('ZIP', 'CITY')

À ce stade, vous pouvez affiner vos données plus manuellement, mais une tâche potentielle de nettoyage des données consiste à supprimer les lignes qui violent les contraintes fonctionnelles entre les colonnes des données, en utilisant la fonction drop_dependency_violations de SemPy.

Pour chaque valeur de la variable déterminante, drop_dependency_violations fonctionne en sélectionnant la valeur la plus courante de la variable dépendante et en supprimant toutes les lignes avec d'autres valeurs. Vous ne devez appliquer cette opération que si vous êtes sûr que cette heuristique statistique conduira aux résultats corrects pour vos données. Sinon, vous devez écrire votre propre code pour gérer les violations détectées selon vos besoins.

In [ ]:
providers_clean = providers.drop_dependency_violations('ZIP', 'CITY')

In [ ]:
providers_clean.list_dependency_violations('ZIP', 'CITY')

La liste vide montre qu'il n'y a plus de violations de la contrainte fonctionnelle CITY -> ZIP.